### 

In [63]:
# make env
# pip install requests
# %pip install pandas
# %pip install matplotlib 
# %pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: 'matplotlib#'

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/40/77/91f92b2fddbd14201bf36cd0c0e7279f1501a88e7a00ef11261c4b95bb7a/scikit_learn-1.4.2-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for scipy>=1.6.0 from https://files.pythonhosted.org/packages/ed/6f/ba2b2f14391291dd47d17da78c3ee644fb3a2fd6bddde664381c1968eda9/scipy-1.13.0-cp312-cp312-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.6 kB ? eta -:--:--
     -------------------------------- ----- 51.2/60.6 kB 650.2 kB/s eta 0:00:01
     -------------------------------------- 60.6/60.6 kB 645.4 kB/s eta 0:00:00
  Obtaining dependency information for joblib>=1.2.0 from https://files.pythonhosted.org/packages/91/29/df4b9b42f2be0b623cbd5e2140cafcaa2bef0759a00b7b70104dcfe2fb51/joblib-1.4.2-py3-none-any.whl.metadata
  Obtaining dependency information for threadpoolctl>=2.0.0 f


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [66]:
# Imports
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

### Load Dataset

In [67]:
dataframes = {}
directory = 'march_mad_data'

# Load general cbb.csv dataset into a dataframe
for filename in os.listdir(directory):
    if filename.endswith("cbb.csv"):
        path = os.path.join(directory, filename)
        dataframes[filename] = pd.read_csv(path)

In [68]:
df = dataframes["cbb.csv"]
print(df)

                TEAM  CONF   G   W  ADJOE  ADJDE  BARTHAG  EFG_O  EFG_D   TOR  \
0     North Carolina   ACC  40  33  123.3   94.9   0.9531   52.6   48.1  15.4   
1          Wisconsin   B10  40  36  129.1   93.6   0.9758   54.8   47.7  12.4   
2           Michigan   B10  40  33  114.4   90.4   0.9375   53.9   47.7  14.0   
3         Texas Tech   B12  38  31  115.2   85.2   0.9696   53.5   43.0  17.7   
4            Gonzaga   WCC  39  37  117.8   86.3   0.9728   56.6   41.1  16.2   
...              ...   ...  ..  ..    ...    ...      ...    ...    ...   ...   
3518          Toledo   MAC  34  27  119.9  109.6   0.7369   56.3   52.9  13.6   
3519         Liberty  ASun  33  27  111.4   97.3   0.8246   55.5   49.3  16.0   
3520     Utah Valley   WAC  34  28  107.1   94.6   0.8065   51.7   44.0  19.3   
3521             UAB  CUSA  38  29  112.4   97.0   0.8453   50.3   47.3  17.3   
3522     North Texas  CUSA  36  31  110.0   93.8   0.8622   51.2   44.5  19.8   

      ...  FTRD  2P_O  2P_D

### Data Cleaning

In [69]:
clean_df = df.replace("N/A", "NA")
print(df["SEED"].unique())
print(f"number of rows: {len(clean_df)}")
print(f"number of columns: {len(clean_df.columns)}")

[ 1.  3.  8.  4.  2.  7.  6.  9. 11. 10.  5. nan 16. 12. 13. 14. 15.]
number of rows: 3523
number of columns: 24


# Drop columns we don't want


### Exploratory Data Analysis (EDA)

In [70]:
seed_df = clean_df.dropna()
print(seed_df["SEED"].unique())
print(len(seed_df))

[ 1.  3.  8.  4.  2.  7.  6.  9. 11. 10.  5. 16. 12. 13. 14. 15.]
680


In [71]:
seed_df = seed_df.drop(columns=['TEAM', 'POSTSEASON'])
one_hot_encoded = pd.get_dummies(seed_df['CONF'])
seed_df = seed_df.drop(columns=['CONF'])
seed_df = pd.concat([seed_df, one_hot_encoded], axis=1)

In [115]:
# Compare conference and whether the team qualifies for March Madness (EDA) !!!!!!!!!!!!!!!!!!!!!

In [72]:
correlation_table = seed_df.corr()['SEED']

from IPython.display import display, HTML

# Sort the correlation table
sorted_correlation_table = correlation_table.sort_values(ascending=False)

# Display the sorted correlation table with scrollable output
display(HTML(sorted_correlation_table.to_frame().to_html()))
# print(correlation_table)

,SEED
SEED,1.000000
ADJDE,0.690730
2P_D,0.369974
EFG_D,0.367349
TOR,0.266311
FTRD,0.214671
NEC,0.188316
SWAC,0.185700
MEAC,0.183084
BSth,0.160213


### EDA Analysis

In [73]:
### Team attributes with the highest impact on seeding 
### (larger magnitude correlations from the correlation table)

# WAB = Wins Above Bubble (May not use because bubble/March Madness cutoff is always changing)
# BARTHAG = Power Ranking / Chance of beating an average D1 team (May not use because subjective power-ranking)
# ADJOE = Adjusted Offensive Efficiency (number of points scored every 100 possessions)
# W = Wins
# G = Games
# ORB = Offensive Rebound Rate
# EFG_O = Effective Field Goal Percentage Short
# 2P_O = 2 Point shooting percentage

# ADJDE = Adjusted Defensive Efficiency (number of points allowed every 100 opponent possessions)
# 2P_D = 2 Point shooting percentage allowed
# EFG_D = Effective Field Goal percentage allowed
# TOR = Turnover Rate
# FTR_D = Free Throw Rate allowed

# B12
# ACC
# B10

### Training Machine Learning Models

In [112]:
# Select the relevant columns for training the model
features = ['G', 'W', 'ADJOE', 'ADJDE', 'EFG_O', 'EFG_D', 'TOR', 'ORB', 'FTRD', '2P_D', 'B12', 'ACC', 'B10']

# Split the data into training and testing sets
X = seed_df[features]
y = seed_df['SEED']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.29411764705882354


In [113]:
import numpy as np

# Get the predicted seed values
predicted_seeds = model.predict(X_test)

# Get the actual seed values
actual_seeds = seed_df.loc[X_test.index]['SEED']

# Calculate the absolute difference between predicted and actual seeds
seed_diff = np.abs(predicted_seeds - actual_seeds)

# Calculate the average distance
average_distance = np.mean(seed_diff)

print("Average Distance between Predicted and Actual Seed:", average_distance)

Average Distance between Predicted and Actual Seed: 1.6470588235294117


In [114]:
# Print actual vs predicted seed to check how accurate the predictions are
predicted_seeds = model.predict(X_test)
for actual_seed, predicted_seed in zip(actual_seeds, predicted_seeds):
    print("Actual Seed:", actual_seed, ", Predicted Seed:", predicted_seed)

Actual Seed: 9.0 , Predicted Seed: 8.0
Actual Seed: 16.0 , Predicted Seed: 16.0
Actual Seed: 3.0 , Predicted Seed: 11.0
Actual Seed: 15.0 , Predicted Seed: 16.0
Actual Seed: 1.0 , Predicted Seed: 3.0
Actual Seed: 16.0 , Predicted Seed: 16.0
Actual Seed: 13.0 , Predicted Seed: 14.0
Actual Seed: 12.0 , Predicted Seed: 10.0
Actual Seed: 12.0 , Predicted Seed: 9.0
Actual Seed: 3.0 , Predicted Seed: 4.0
Actual Seed: 7.0 , Predicted Seed: 7.0
Actual Seed: 3.0 , Predicted Seed: 5.0
Actual Seed: 15.0 , Predicted Seed: 15.0
Actual Seed: 16.0 , Predicted Seed: 16.0
Actual Seed: 6.0 , Predicted Seed: 7.0
Actual Seed: 8.0 , Predicted Seed: 12.0
Actual Seed: 9.0 , Predicted Seed: 9.0
Actual Seed: 9.0 , Predicted Seed: 4.0
Actual Seed: 11.0 , Predicted Seed: 9.0
Actual Seed: 16.0 , Predicted Seed: 16.0
Actual Seed: 2.0 , Predicted Seed: 1.0
Actual Seed: 10.0 , Predicted Seed: 4.0
Actual Seed: 9.0 , Predicted Seed: 11.0
Actual Seed: 15.0 , Predicted Seed: 14.0
Actual Seed: 15.0 , Predicted Seed: 13.0

In [105]:
# Predict whether a team will qualify for March Madness

In [108]:
# Predict final post season result

In [ ]:
# Run on cbb24.csv